# 패키지 가져오기

In [1]:
import pandas as pd
import numpy as np
import random
import os

import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from tqdm.notebook import tqdm

In [40]:
def set_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds(777)

# 디렉토리 설정

In [3]:
os.chdir('/content/drive/MyDrive/Competitions/human_ai')
os.getcwd()

'/content/drive/MyDrive/Competitions/human_ai'

# 데이터 전처리

## 가져오기

In [4]:
train_org = pd.read_csv('dataset/train.csv')
test_org = pd.read_csv('dataset/test.csv')

In [5]:
train_org.head()

,id,sentence1,sentence2,sentence3,sentence4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1


## Train 데이터 변형
- multiple classification 문제를 binary classification 문제로 치환

In [6]:
train_org = train_org.rename(columns={key:key[-1] for key in [f'sentence{i}' for i in range(1, 5)]})
train_org.head()

,id,1,2,3,4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1


In [7]:
train_data = train_org.melt(id_vars='label', value_vars=['1', '2', '3', '4'], value_name='sentence')

tmp = train_data.apply(lambda x: 1 if int(x.label) == int(x.variable) else 0, axis=1)

train_data['target'] = tmp
train_data = train_data[['sentence', 'target']]
train_data

,sentence,target
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",0
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",1
...,...,...
195,ㄹㅇ남자직원 왜케 말투 공격적인거에요? 밥먹고 기분좋게 카페와서 커피먹을라고 했는데...,1
196,"이 가격에 이런 품질의 피자라니, 너무 기분이 좋아요! 도우는 매우 쫄깃하고 고구마...",0
197,"겉은 바삭바삭한데, 치킨 메뉴가 많고 주인부부가 인상적이어서 들어갔는데 바로 나왔어...",0
198,"단체로 와서 잘먹긴했지만 회만 먹은것도 아니고 술도 시키면서 먹고있는데 ""적당히""드...",1


# Tokenizer / Language Model 가져오기

In [9]:
MODEL_NAME = 'kykim/funnel-kor-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
pretrained = AutoModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Dataset / Dataloader

## Train Dataset / Train DataLoader 선언

In [26]:
class TrainDataset(Dataset):
    def __init__(self, df, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        token_result = self.tokenizer(self.df.iloc[idx, 0], padding='max_length', truncation=True, max_length=self.max_length, add_special_tokens=True, return_tensors='pt')
        return [token_result['input_ids'][0], token_result['token_type_ids'][0], token_result['attention_mask'][0]], self.df.iloc[idx, 1]

## Train/Evaluation Split

In [41]:
train_df, valid_df = train_test_split(train_data, test_size=0.4, random_state=777, stratify=train_data['target'])
# valid_df, test_df = train_test_split(valid_df, test_size=0.5, random_state=11, stratify=valid_df['target'])

In [42]:
print(len(train_df))
print(len(valid_df))
# print(len(test_df))

120
80


In [62]:
train_dataset = TrainDataset(train_df, max_length=312)
valid_dataset = TrainDataset(valid_df, max_length=312)
# test_dataset = TrainDataset(test_df, max_length=312)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# Model 선언 + SwiGLU

In [55]:
class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1) # 마지막 dimension에 대해 절반으로 나눔
        return F.silu(gate) * x # SiLU는 Swish의 beta=1

class Classifier(nn.Module):
    def __init__(self, model):
        super(Classifier, self).__init__()
        self.model = model
        self.classifier = nn.Sequential(
            SwiGLU(),
            nn.Linear(384, 1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input_ids, token_type_ids, attention_mask):
        vector = self.model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        output = self.classifier(vector.last_hidden_state[:, 0, :].detach())
        return output

# 학습

## 헬퍼 함수

In [67]:
def train(model, train_loader, valid_loader, optimizer, loss_fn, scheduler, device='cuda:0'):
    print('[[[[ Train ]]]]')

    model.train()

    preds = []
    answers = []
    losses = []

    for [input_ids, token_type_ids, attention_mask], labels in tqdm(train_loader):
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.view(-1).to(device)

        #### optimizer 초기화
        optimizer.zero_grad()

        #### 모델 연산 결과
        output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).view(-1)

        #### score 산정을 위한 정답/예측값 리스트 취합
        # predicted = (output > 0).int().tolist()
        predicted = output.round().int().tolist()
        preds.extend(predicted) # 예측값
        answers.extend(labels.int().tolist()) # 정답

        #### loss 계산
        loss = loss_fn(output.float(), labels.float())
        loss.backward()
        losses.append(loss.item()) # loss값 저장

        #### 최적화
        optimizer.step()

        #### 디버깅용 프린트
        # print('y_hat : ', output.round(decimals=4).tolist()) # 모델 연산 결과
        # print('preds : ', predicted) # 예측 결과
        # print('labels: ', labels.tolist()) # 정답

    #### 성능 기록
    acc = accuracy_score(answers, preds)
    f1 = f1_score(answers, preds)
    avg_loss = np.mean(losses)

    print(f'accuracy: {acc:.6f}, f1-score: {f1:.6f}, loss:{avg_loss:.6f}')
    print()

    eval_loss, eval_acc, eval_f1 = evaluate(model, valid_loader, loss_fn, device)

    #### 스케줄러로 LR 조정
    scheduler.step(eval_loss)

    #### loss, accuracy, f1-score 반환
    return avg_loss, acc, f1

In [68]:
def evaluate(model, valid_loader, loss_fn, device='cuda:0'):
    print('[[[[ Evaluation ]]]]')

    model.eval()

    preds = []
    answers = []
    losses = []

    for [input_ids, token_type_ids, attention_mask], labels in tqdm(train_loader):
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.view(-1).to(device)

        #### 모델 연산 결과
        output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).view(-1)

        #### score 산정을 위한 정답/예측값 리스트 취합
        # predicted = (output > 0).int().tolist()
        predicted = output.round().int().tolist()
        preds.extend(predicted) # 예측값
        answers.extend(labels.int().tolist()) # 정답

        #### loss 계산
        loss = loss_fn(output.float(), labels.float())
        # loss.backward()
        losses.append(loss.item()) # loss값 저장

        #### 디버깅용 프린트
        # print('y_hat : ', output.round(decimals=4).tolist()) # 모델 연산 결과
        # print('preds : ', predicted) # 예측 결과
        # print('labels: ', labels.tolist()) # 정답

    #### 성능 기록
    acc = accuracy_score(answers, preds)
    f1 = f1_score(answers, preds)
    avg_loss = np.mean(losses)

    print(f'accuracy: {acc:.6f}, f1-score: {f1:.6f}, loss:{avg_loss:.6f}')
    print()

    #### loss, accuracy, f1-score 반환
    return avg_loss, acc, f1

## 시작

In [70]:
gc.collect()
torch.cuda.empty_cache()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
N_EPOCHS = 20

model = Classifier(pretrained).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):
    print(f'### EPOCH {epoch+1} ###')

    loss, acc, f1 = train(model, train_loader, valid_loader, optimizer, loss_fn, scheduler, device)

    print('=' * 50)
    print()

### EPOCH 1 ###
[[[[ Train ]]]]


  0%|          | 0/8 [00:00<?, ?it/s]

accuracy: 0.691667, f1-score: 0.244898, loss:0.576226

[[[[ Evaluation ]]]]


  0%|          | 0/8 [00:00<?, ?it/s]

accuracy: 0.750000, f1-score: 0.000000, loss:0.494510



### EPOCH 2 ###
[[[[ Train ]]]]


...

accuracy: 0.933333, f1-score: 0.846154, loss:0.169750



### EPOCH 20 ###
[[[[ Train ]]]]


  0%|          | 0/8 [00:00<?, ?it/s]

accuracy: 0.916667, f1-score: 0.807692, loss:0.229294

[[[[ Evaluation ]]]]


  0%|          | 0/8 [00:00<?, ?it/s]

accuracy: 0.925000, f1-score: 0.823529, loss:0.180232





# Submission

## Submission 데이터 변형

In [72]:
submit_df = test_org.melt(id_vars='id', value_vars=['sentence1', 'sentence2', 'sentence3', 'sentence4']).sort_values(['id', 'variable']).reset_index(drop=True).loc[:, ['value']]
submit_df.columns = ['sentence']
submit_df.head()

,sentence
0,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매..."
1,"배송은 정말 빠르고요~ 집에서 속옷은 불편해서 안 입으려고 했는데, 약간 부끄럽기도..."
2,배송은 빠르구여~집에서 속옷 불편한데 입긴싫고~안입자니 민망해서 구매했는데~오 사이...
3,배송 빨라요~ 속옷 불편한데 입기 싫고~ 안 입으면 민망해서 삿는데~ 사이즈 크네요...
4,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...


## Submission Dataset / Submission DataLoader 선언

In [75]:
class SubmitDataset(Dataset):
    def __init__(self, df, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        token_result = self.tokenizer(self.df.iloc[idx, 0], padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return token_result['input_ids'][0], token_result['token_type_ids'][0], token_result['attention_mask'][0]

In [76]:
submit_dataset = SubmitDataset(submit_df, max_length=312)
submit_loader = DataLoader(submit_dataset, batch_size=4, shuffle=False)

In [ ]:
model.eval()

submission_answers = []
for input_ids, token_type_ids, attention_mask in tqdm(submit_loader):
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_mask = attention_mask.to(device)

    output = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).view(-1)
    predicted = (output.argmax().int() + 1).tolist()
    submission_answers.append(predicted)

submission_answers

  0%|          | 0/1100 [00:00<?, ?it/s]